In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

在这个notebook中，使用相同的填充和归一化方法、

即数据的特征仍不做处理，

在这样特原始条件下比较不同模型的效果

将尝试LR，决策树，adaboost，随机森林，SVM，GMM，KNN，等，参见titanic的notebook


# 1，读入数据

In [ ]:
train = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv',index_col=0)
test = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv',index_col=0)
sample = pd.read_csv('/kaggle/input/GiveMeSomeCredit/sampleEntry.csv')

train.shape, test.shape, sample.shape


# 2， 填充缺失值

In [ ]:
train_default_dict = {} 
for attr in ['MonthlyIncome', 'NumberOfDependents']:
    print(attr, end='\t')
    median, mean, mode = train[attr].median(), train[attr].mean(), train[attr].mode()
    mode = mode.values
    if len(mode) == 1:
        mode = mode[0]
    train_default_dict[attr] = [median, mean, mode]
    print(median, mean, mode)
train_default_dict


In [ ]:
test_default_dict = {} 
for attr in ['MonthlyIncome', 'NumberOfDependents']:
    print(attr, end='\t')
    median, mean, mode = test[attr].median(), test[attr].mean(), test[attr].mode()
    mode = mode.values
    if len(mode) == 1:
        mode = mode[0]
    test_default_dict[attr] = [median, mean, mode]
    print(median, mean, mode)
test_default_dict


In [ ]:
train.MonthlyIncome.fillna(train_default_dict['MonthlyIncome'][0], inplace=True)
test.MonthlyIncome.fillna(train_default_dict['MonthlyIncome'][0], inplace=True)
train.NumberOfDependents.fillna(test_default_dict['NumberOfDependents'][1], inplace=True)
test.NumberOfDependents.fillna(test_default_dict['NumberOfDependents'][1], inplace=True)

train.isnull().sum(), '*'*50, test.isnull().sum()


# 3， 分离X_train, y_train, X_test

In [ ]:
X_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values
X_test = test.iloc[:,1:].values
X_train.shape, y_train.shape, X_test.shape

# 4, 分别归一化X_train, X_test 

In [ ]:
from sklearn import preprocessing

train_scaler = preprocessing.StandardScaler().fit(X_train)
print( train_scaler.mean_ , '\n'+'-'*50+'\n', train_scaler.scale_)
print('='*50)
test_scaler = preprocessing.StandardScaler().fit(X_test)
print( test_scaler.mean_ , '\n'+'-'*50+'\n', test_scaler.scale_)


In [ ]:
X_train_scaled = train_scaler.transform(X_train)
X_test_scaled = test_scaler.transform(X_test)

X_train_scaled.mean(axis=0), X_train_scaled.std(axis=0),   X_test_scaled.mean(axis=0), X_test_scaled.std(axis=0)


# 5，数据内容不再改动，比较模型
## 首先从X_train, y_train划分出训练集X_learn, y_learn; 验证集 X_valid, y_valid


In [ ]:
#比较不同模型的适合程度，需要划分train 和 valid 集合
#使用sklearn model_selection
#简单点 使用 train_test_split
#这里尝试使用 KFold, 不好操作

In [ ]:
from sklearn.model_selection import train_test_split

X_learn, X_valid, y_learn, y_valid = train_test_split(X_train_scaled, y_train, random_state=0)
X_learn.shape, X_valid.shape, y_learn.shape, y_valid.shape

In [ ]:
#使用roc_auc 作为 metric
from sklearn.metrics import roc_auc_score

In [ ]:
# 用来最后比较各种方法的表格 form
form = []

## 5.1 逻辑回归

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
#之前没有注意 linear Regression 和 Logistic Regression 的区别，后者是前者的输出经过sigmiod
#但由于 sigmoid 是单调增函数， 所以不影响 ROC_AUC 的值，这里只用 Logistic Regression 即可

arg, maxauc = 'none', 0
for s in ['newton-cg', 'lbfgs', 'liblinear']:
    model = LogisticRegressionCV(scoring='roc_auc', solver=s)
    model.fit(X_learn, y_learn)
    y_pred = model.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    print(s, score)
    if score > maxauc:
        arg, maxauc = s, score
print()
print(arg, maxauc)
form.append(['LogisticRegressionCV_'+arg,maxauc])

## 5.2 支持向量机
*  放弃，用时太长
*  使用前5000个样本训练，粗略构造分类平面
*  意外发现linear kernel在X_valid上效果很好

In [ ]:
from sklearn.svm import SVC
# 训练 时间过长
# 仅学习前 5000 还可以， 6000 训练时间都开始变长
#并且前5000 在 X_valid 上效果不错

arg, maxauc = 'none', 0
for k in ['linear', 'poly', 'rbf', 'sigmoid']:
    svc = SVC(kernel=k, probability=True)
    svc.fit(X_learn[:5000], y_learn[:5000])
    y_pred = svc.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    print(k, score)
    if score > maxauc:
        arg, maxauc = k, score
print()
print(arg, maxauc)
form.append(['SVC_'+arg ,maxauc])

## 5.3 KNN
略逊于 LR?\
调整N_neighbors!

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import itertools

arglist = itertools.product(['uniform', 'distance'],['auto', 'ball_tree', 'kd_tree', 'brute'])
print(arglist,end='\n*******\n')

weight, algorithm, maxauc = '', '', 0
for wei, algo in arglist:
    knn = KNeighborsClassifier(weights=wei, algorithm=algo)
    knn.fit(X_learn, y_learn)
    y_pred = knn.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    print(wei, algo, score)
    if score > maxauc:
        weight, algorithm, maxauc = wei, algo, score
print()
print(weight, algorithm, maxauc)

* distance auto 为最优参数
* 对 n_neighbors进行调参

In [ ]:
scorelist = []
n_neighbors, maxauc = -1, 0
for k in range(100, 1000+1, 100):
    knn = KNeighborsClassifier(n_neighbors=k, weights=weight, algorithm=algorithm)
    knn.fit(X_learn, y_learn)
    y_pred = knn.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    print(k, score)
    if score > maxauc:
        n_neighbors, maxauc = k, score
    scorelist.append(score)
print()
print(n_neighbors, maxauc)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(100, 1000+1, 100),scorelist)
plt.title('score - n_neighbors')
plt.xlabel('n_neighbors')
plt.ylabel('score(AUC)')

In [ ]:
scorelist = []
n_neighbors, maxauc = -1, 0
for k in range(200, 500+1, 50):
    knn = KNeighborsClassifier(n_neighbors=k, weights=weight, algorithm=algorithm)
    knn.fit(X_learn, y_learn)
    y_pred = knn.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    print(k, score)
    if score > maxauc:
        n_neighbors, maxauc = k, score
    scorelist.append(score)
print()
print(n_neighbors, maxauc)

In [ ]:
plt.plot(range(200, 500+1, 50),scorelist)
plt.title('score - n_neighbors')
plt.xlabel('n_neighbors')
plt.ylabel('score(AUC)')

In [ ]:
scorelist = []
n_neighbors, maxauc = -1, 0
for k in range(320, 400+1, 10):
    knn = KNeighborsClassifier(n_neighbors=k, weights=weight, algorithm=algorithm)
    knn.fit(X_learn, y_learn)
    y_pred = knn.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    print(k, score)
    if score > maxauc:
        n_neighbors, maxauc = k, score
    scorelist.append(score)
print()
print(n_neighbors, maxauc)
form.append(['KNN_{}_{}_{}'.format(weight,algorithm,n_neighbors),maxauc])

## 5.4 （高斯）朴素贝叶斯

In [ ]:
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(X_learn, y_learn)
y_pred = gaussian.predict_proba(X_valid)[:,1]
score = roc_auc_score(y_valid, y_pred)
print(score)

form.append(['GaussianNB',score])

## 5.5 感知机
* 没有概率值作为输出很难获得好的AUC,单层效果本来也不够好
* 借助MLP，实现概率预测
* 3层就够了

In [ ]:
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(X_learn, y_learn)
y_pred = perceptron.predict(X_valid)
score = roc_auc_score(y_valid, y_pred)
print(score)

In [ ]:
from sklearn.neural_network import MLPClassifier

scorelist = []
best_layer, maxauc = -1, 0
for l in range(1,5):
    mlp = MLPClassifier(hidden_layer_sizes=l)
    mlp.fit(X_learn, y_learn)
    y_pred = mlp.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    print(l, score)
    if score > maxauc:
        best_layer, maxauc = l, score
    scorelist.append(score)
print()
print(best_layer, maxauc)
form.append(['MLPClassifier_{}'.format(best_layer),score])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1,5),scorelist)

## 5.6 linearSVC
和SVC有什么区别？ 快？
* 无法收敛,即使增加了迭代次数
* 无法 predict_proba

In [ ]:
# from sklearn.svm import LinearSVC

# linear_svc = LinearSVC()
# linear_svc.fit(X_learn, y_learn)
# y_pred = linear_svc.predict(X_valid)
# score = roc_auc_score(y_valid, y_pred)
# print(score)

In [ ]:
# for iternum in [1000, 5000, 10000]:
    
#     linear_svc = LinearSVC(max_iter=iternum,)
#     linear_svc.fit(X_learn, y_learn)
#     y_pred = linear_svc.predict(X_valid)
#     score = roc_auc_score(y_valid, y_pred)
#     print(score)
# #增加时间，效果没有提升.仍然不收敛

## 5.7 SGDClassifier
minibatch而已,快

In [ ]:
from sklearn.linear_model import SGDClassifier
import itertools

# args for svm
arglist = itertools.product(['hinge', 'log','modified_huber','squared_hinge','perceptron'],
                            ['l2', 'l1', 'elasticnet'])

loss, penalty, maxauc = 'none', 'none', 0
for l, p in arglist:
    sgd = SGDClassifier(loss=l, penalty=p)
    sgd.fit(X_learn, y_learn)
    y_pred = sgd.predict(X_valid)
    score = roc_auc_score(y_valid, y_pred)
    print(l, p, score)
    if score > maxauc:
        loss, penalty, maxauc = l, p, score
print()
print(loss, penalty, maxauc)

In [ ]:
#args for logistic regresion
arglist = itertools.product(['squared_loss','huber', 'epsilon_insensitive','squared_epsilon_insensitive'],
                            ['l2', 'l1', 'elasticnet'])

for l, p in arglist:
    sgd = SGDClassifier(loss=l, penalty=p)
    sgd.fit(X_learn, y_learn)
    y_pred = sgd.predict(X_valid)
    score = roc_auc_score(y_valid, y_pred)
    print(l, p, score)
    if score > maxauc:
        loss, penalty, maxauc = l, p, score
print()
print(loss, penalty, maxauc)

form.append(['SGDClassifier_{}_{}'.format(loss,penalty),maxauc])

相比之下，svm中的square hinge作为loss具有很好的auc，比其他方法高出约0.2

## 5.8 决策树

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid={
        'criterion':['gini','entropy']
    },
    scoring='roc_auc',
    verbose=3
)
grid.fit(X_learn, y_learn)


In [ ]:
for result in grid.cv_results_:
    print(result, grid.cv_results_[result])
grid.best_params_['criterion']

In [ ]:
tree = DecisionTreeClassifier(criterion=grid.best_params_['criterion'])
tree.fit(X_learn, y_learn)
y_pred = tree.predict_proba(X_valid)[:,1]
score = roc_auc_score(y_valid, y_pred)
print(score)
form.append(['DecisionTreeClassifier_{}'.format(grid.best_params_['criterion']),score])

## 5.9 随机森林
* 随机森林的结果，不会差
* 就是网格搜索+交叉验证太慢

In [ ]:
from sklearn.ensemble import RandomForestClassifier

grid = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid={
        'n_estimators':[30,50,80,100,200]
    },
    scoring='roc_auc',
    verbose=3
)

grid.fit(X_learn, y_learn)
for result in grid.cv_results_:
    print(result, grid.cv_results_[result])
grid.best_params_['n_estimators']



In [ ]:
rfc = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'])
rfc.fit(X_learn, y_learn)
y_pred = rfc.predict_proba(X_valid)[:,1]
score = roc_auc_score(y_valid, y_pred)
print(score)

form.append(['RandomForestClassifier_{}'.format(grid.best_params_['n_estimators']),score])

## 5.10 GDBT之Lightgbm

In [ ]:
import lightgbm as lgb

grid = GridSearchCV(
    estimator=lgb.LGBMClassifier(),
    param_grid={
        'n_estimators':range(40,60)
    },
    scoring='roc_auc',
    verbose=1
)

grid.fit(X_learn, y_learn)
for result in grid.cv_results_:
    print(result, grid.cv_results_[result])
grid.best_params_['n_estimators']

In [ ]:
clf = lgb.LGBMClassifier(n_estimators=grid.best_params_['n_estimators'])
clf.fit(X_learn, y_learn)
y_pred = clf.predict_proba(X_valid)[:,1]
score = roc_auc_score(y_valid, y_pred)
print(score)

form.append(['LGBMClassifier_{}'.format(grid.best_params_['n_estimators']),score])

In [ ]:
form.sort(key=lambda x:x[1],reverse=True)
form

In [ ]:
fm = pd.DataFrame(form,columns=['Model','AUC'])
fm

# 6 针对最优模型调参
lr仍然是0.1最优，n还是选取54

In [ ]:
grid = GridSearchCV(
    estimator=lgb.LGBMClassifier(),
    param_grid={
        'n_estimators':range(45,55),
        'learning_rate':[0.05,0.1,0.15,0.2,0.25,0.3]
    },
    scoring='roc_auc',
    verbose=1
)
grid.fit(X_learn,y_learn)

In [ ]:
grid.best_params_['n_estimators'], grid.best_params_['learning_rate']


# 7 生成结果 制作文件

In [ ]:
clf = lgb.LGBMClassifier(n_estimators=grid.best_params_['n_estimators'],
                         learning_rate=grid.best_params_['learning_rate'])
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)[:,1]


In [ ]:
sample = pd.read_csv('../input/GiveMeSomeCredit/sampleEntry.csv')
sample

In [ ]:
sample['Probability'] = y_pred
sample.to_csv('./submit.csv',index=False)
reload = pd.read_csv('./submit.csv')
reload